In [2]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import json
import os
from datetime import datetime
from scrapers.bergfreunde import get_price

In [3]:

app = FastAPI()

# Allow frontend (e.g., GitHub Pages) to fetch from this API
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

PRICES_FILE = "prices.json"

# Load stored prices or initialize
def load_prices():
    if not os.path.exists(PRICES_FILE):
        return {}
    with open(PRICES_FILE, "r") as f:
        return json.load(f)

def save_prices(prices):
    with open(PRICES_FILE, "w") as f:
        json.dump(prices, f, indent=2)

@app.get("/products")
def get_products():
    current = get_price()
    current_price = current["price"]
    now = datetime.now().isoformat()

    prices = load_prices()
    product_key = "bergfreunde"

    previous = prices.get(product_key, {})
    previous_price = previous.get("price", current_price)

    dropped = current_price < previous_price

    # Save updated price
    prices[product_key] = {
        "price": current_price,
        "last_checked": now
    }
    save_prices(prices)

    return [{
        "name": current["title"],
        "url": current["url"],
        "price": current_price,
        "previous_price": previous_price,
        "dropped": dropped,
        "last_checked": now
    }]


In [4]:
get_products()

Final price 2: 166,46


[{'name': 'Indalo - Klimschoenen',
  'url': 'https://www.bergfreunde.nl/tenaya-indalo-klimschoenen/?aid=6e824d326b73d7b3ceccfc29515ebf76&pid=10004&bfgb=false',
  'price': '166,46',
  'previous_price': '166,46',
  'dropped': False,
  'last_checked': '2025-04-08T12:11:23.155508'}]